In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/crolling_final.csv')

In [4]:
!pip install kmodes scikit-learn

In [5]:
import pandas as pd
from kmodes.kmodes import KModes
from sklearn.preprocessing import MultiLabelBinarizer
data = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/crolling_final.csv')
# 데이터 불러오기
# 제외할 혜택 리스트
exclude_categories = [
    '연회비지원', '프리미엄', '렌탈', '오토', '금융', '바우처',
    '국민행복카드', 'Priority Pass', '경차유류환급', '그린카드', '체크카드겸용','언제나할인'
]
# 'Category_List' 열 생성: 'Benefits'에서 'Category' 정보 추출 및 제외할 항목 제거
data['Category_List'] = data['Benefits'].apply(
    lambda x: [item for item in eval(x)[0]['Category'] if item not in exclude_categories]
    if isinstance(x, str) and eval(x) and 'Category' in eval(x)[0] else []
)
# 빈 리스트가 있는 행 제거
data = data[data['Category_List'].map(len) > 0]
# MultiLabelBinarizer를 사용해 'Category_List'를 One-hot 인코딩
mlb = MultiLabelBinarizer()
category_encoded = mlb.fit_transform(data['Category_List'])
# K-mode 클러스터링 수행 (예: 4개 클러스터로 설정)
kmode = KModes(n_clusters=3, init="Cao", n_init=1, verbose=1, random_state=42)
clusters = kmode.fit_predict(category_encoded)
# 클러스터 결과를 데이터프레임에 추가
data['Cluster'] = clusters
# 각 클러스터별 카드 개수 확인
card_count_per_cluster = data['Cluster'].value_counts()
print("Cluster별 카드 개수:\n", card_count_per_cluster)
# 클러스터별로 혜택 분포 요약
cluster_summary = data.groupby('Cluster')['Category_List'].apply(lambda x: x.explode().value_counts()).unstack().fillna(0)
print("Cluster별 혜택 요약:\n", cluster_summary)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 21, cost: 509.0
Cluster별 카드 개수:
 Cluster
0    109
2     29
1     28
Name: count, dtype: int64
Cluster별 혜택 요약:
          간편결제  관리비    교육  납부 혜택  대중교통  대형마트    레저    문화  반려동물    뷰티  ...  \
Cluster                                                              ...   
0        14.0  9.0   6.0    8.0  16.0  16.0  12.0  13.0   1.0   4.0  ...   
1        16.0  2.0  27.0    1.0  18.0  28.0  25.0  23.0   9.0  25.0  ...   
2         5.0  3.0   2.0    0.0  18.0   6.0   7.0  17.0   1.0   4.0  ...   

           외식    육아    의료    주유  카페/베이커리    통신   편의점  포인트/캐시백  하이패스  항공마일리지  
Cluster                                                                      
0        13.0   0.0   8.0  30.0     23.0  18.0   8.0     43.0   2.0    22.0  
1        28.0  20.0  25.0  27.0     27.0  27.0  25.0     14.0   3.0     1.0  
2        20.0   0.0   4.0   7.0     26.0  18.0  24.0      5.0   1.0     0.0  

[3 ro

In [6]:
data

,Unnamed: 0,Card Name,Annual_fee,Base Record,Benefits,Category_List,Cluster
0,0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,"[{'Category': ['항공마일리지', '언제나할인'], 'Title': ['...",[항공마일리지],0
1,1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[{'Category': ['연회비지원', '언제나할인', '쇼핑', '레저', '...","[쇼핑, 레저, 카페/베이커리, 편의점, 대형마트, 의료, 뷰티, 대중교통, 외식]",2
2,2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[{'Category': ['프리미엄', '쇼핑', '주유', '카페/베이커리', ...","[쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트, 외식, 통...",1
3,3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[{'Category': ['연회비지원', '언제나할인', '대중교통', '간편결제...","[대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유, 외식, 쇼...",1
4,4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[{'Category': ['연회비지원', '포인트/캐시백', '카페/베이커리', ...","[포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]",2
...,...,...,...,...,...,...,...
293,293,LOCA PLATINUM(할인형),"국내 10만원, 해외 10만원, 가족겸용 없음",조건없음,"[{'Category': ['언제나할인', '바우처', '프리미엄', '카페/베이커...","[카페/베이커리, 쇼핑, 뷰티, 대형마트, 편의점, 주유, 외식, 영화, 통신, 교...",1
294,294,I-MILEAGE(아시아나),"국내 4만1천원, 해외 4만2천원",직전 1개월 합계 50만원 이상,"[{'Category': ['항공마일리지', '프리미엄', '카페/베이커리', '연...","[항공마일리지, 카페/베이커리]",0
299,299,The CJ-현대카드M Edition2,"국내 1만5천원, 해외 2만원",조건없음,"[{'Category': ['쇼핑', '영화', '외식', '카페/베이커리', '포...","[쇼핑, 영화, 외식, 카페/베이커리, 포인트/캐시백]",0
300,300,KB국민 그린카드(서울형),"국내전용 없음, 해외겸용 없음",직전 1개월 합계 20만원 이상,"[{'Category': ['포인트/캐시백', '언제나할인', '대중교통', '쇼핑...","[포인트/캐시백, 대중교통, 쇼핑, 대형마트]",0


In [ ]:
import pandas as pd
from kmodes.kmodes import KModes
from sklearn.preprocessing import MultiLabelBinarizer
# 데이터 불러오기
# data = pd.read_csv('crolling_final.csv')
# Category 열 추출 및 인코딩
# data['Category_List'] = data['Benefits'].apply(lambda x: eval(x)[0]['Category'] if isinstance(x, str) and eval(x) and 'Category' in eval(x)[0] else [])

# 'Category_List' 열 생성: 'Benefits'에서 'Category' 정보 추출 및 '언제나할인' 제거
data['Category_List'] = data['Benefits'].apply(
    lambda x: [item for item in eval(x)[0]['Category'] if item != '언제나할인']
    if isinstance(x, str) and eval(x) and 'Category' in eval(x)[0] else []
)
# 빈 리스트가 있는 행 제거
data = data[data['Category_List'].map(len) > 0]
data

,Unnamed: 0,Card Name,Annual_fee,Base Record,Benefits,Category_List,Cluster
0,0,하나 트래블로그 SKYPASS 신용카드,"국내 4만8천원, 해외 4만8천원, 가족겸용 없음",조건없음,"[{'Category': ['항공마일리지', '언제나할인'], 'Title': ['...",[항공마일리지],0
1,1,DA카드의정석Ⅱ,"국내 1만2천원, 해외 1만2천원",직전 1개월 합계 30만원 이상,"[{'Category': ['연회비지원', '언제나할인', '쇼핑', '레저', '...","[연회비지원, 쇼핑, 레저, 프리미엄, 카페/베이커리, 편의점, 대형마트, 의료, ...",2
2,2,삼성카드 & MILEAGE PLATINUM(스카이패스),"국내 4만7천원, 해외 4만9천원",조건없음,"[{'Category': ['프리미엄', '쇼핑', '주유', '카페/베이커리', ...","[프리미엄, 쇼핑, 주유, 카페/베이커리, 편의점, 대중교통, 간편결제, 대형마트,...",1
3,3,현대카드ZERO Edition3(할인형),"국내 1만5천원, 해외 1만5천원, 가족 5천원",조건없음,"[{'Category': ['연회비지원', '언제나할인', '대중교통', '간편결제...","[연회비지원, 대중교통, 간편결제, 편의점, 통신, 카페/베이커리, 대형마트, 주유...",1
4,4,삼성카드 taptap O,"국내 1만원, 해외 1만원",직전 1개월 합계 30만원 이상,"[{'Category': ['연회비지원', '포인트/캐시백', '카페/베이커리', ...","[연회비지원, 포인트/캐시백, 카페/베이커리, 쇼핑, 통신, 대중교통, 영화, 편의점]",2
...,...,...,...,...,...,...,...
293,293,LOCA PLATINUM(할인형),"국내 10만원, 해외 10만원, 가족겸용 없음",조건없음,"[{'Category': ['언제나할인', '바우처', '프리미엄', '카페/베이커...","[바우처, 프리미엄, 카페/베이커리, 쇼핑, 뷰티, 대형마트, 편의점, 주유, 외식...",1
294,294,I-MILEAGE(아시아나),"국내 4만1천원, 해외 4만2천원",직전 1개월 합계 50만원 이상,"[{'Category': ['항공마일리지', '프리미엄', '카페/베이커리', '연...","[항공마일리지, 프리미엄, 카페/베이커리, 연회비지원]",0
299,299,The CJ-현대카드M Edition2,"국내 1만5천원, 해외 2만원",조건없음,"[{'Category': ['쇼핑', '영화', '외식', '카페/베이커리', '포...","[쇼핑, 영화, 외식, 카페/베이커리, 포인트/캐시백]",0
300,300,KB국민 그린카드(서울형),"국내전용 없음, 해외겸용 없음",직전 1개월 합계 20만원 이상,"[{'Category': ['포인트/캐시백', '언제나할인', '대중교통', '쇼핑...","[포인트/캐시백, 대중교통, 쇼핑, 대형마트, 그린카드]",0


In [7]:
data = data.drop('Benefits',axis=1)

In [10]:
data=data.drop(['Cluster','Unnamed: 0'],axis=1).reset_index(drop=True)

In [ ]:

mlb = MultiLabelBinarizer()
category_encoded = mlb.fit_transform(data['Category_List'])
# K-mode 클러스터링 수행
kmeans = KModes(n_clusters=4, init="Cao", n_init=5, verbose=1, random_state=42)
clusters = kmeans.fit_predict(category_encoded)
# 클러스터 결과 추가 및 클러스터 요약 표시
data['Cluster'] = clusters
cluster_summary = data.groupby('Cluster')['Category_List'].apply(lambda x: x.explode().value_counts()).unstack().fillna(0)
print(cluster_summary)

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 9, cost: 703.0
Run 1, iteration: 2/100, moves: 10, cost: 690.0
Run 1, iteration: 3/100, moves: 9, cost: 681.0
Run 1, iteration: 4/100, moves: 2, cost: 681.0
         Priority Pass  간편결제  경차유류환급  관리비    교육  국민행복카드  그린카드    금융  납부 혜택  \
Cluster                                                                      
0                  3.0  13.0     2.0  7.0   4.0     1.0   2.0   7.0    8.0   
1                  0.0  16.0     0.0  2.0  26.0     1.0   0.0  10.0    1.0   
2                  0.0   1.0     0.0  3.0   3.0     1.0   0.0   3.0    0.0   
3                  0.0   5.0     0.0  2.0   2.0     0.0   0.0   1.0    0.0   

         대중교통  ...    의료    주유  체크카드겸용  카페/베이커리    통신   편의점  포인트/캐시백  프리미엄  \
Cluster        ...                                                           
0        15.0  ...   7.0  27.0

In [12]:
data.to_csv('card_data.csv',index=False)

In [ ]:
cluster_summary.head()

,Priority Pass,간편결제,경차유류환급,관리비,교육,국민행복카드,그린카드,금융,납부 혜택,대중교통,...,의료,주유,체크카드겸용,카페/베이커리,통신,편의점,포인트/캐시백,프리미엄,하이패스,항공마일리지
Cluster,,,,,,,,,,,,,,,,,,,,,
0,3.0,14.0,2.0,7.0,4.0,1.0,2.0,7.0,8.0,18.0,...,7.0,30.0,2.0,26.0,16.0,12.0,43.0,29.0,3.0,22.0
1,0.0,16.0,0.0,1.0,26.0,1.0,0.0,10.0,1.0,17.0,...,25.0,26.0,0.0,27.0,26.0,25.0,14.0,8.0,3.0,1.0
2,0.0,4.0,0.0,3.0,2.0,0.0,0.0,1.0,0.0,15.0,...,4.0,7.0,0.0,22.0,18.0,20.0,5.0,4.0,0.0,0.0
3,0.0,1.0,0.0,3.0,3.0,1.0,0.0,3.0,0.0,2.0,...,1.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import pandas as pd
from kmodes.kmodes import KModes
from sklearn.preprocessing import MultiLabelBinarizer

# 'Category' 정보 추출 및 인코딩
data['Category_List'] = data['Benefits'].apply(lambda x: eval(x)[0]['Category'] if isinstance(x, str) and eval(x) and 'Category' in eval(x)[0] else [])
mlb = MultiLabelBinarizer()
category_encoded = mlb.fit_transform(data['Category_List'])
# K-mode 클러스터링 수행
kmeans = KModes(n_clusters=3, init="Cao", n_init=1, verbose=0, random_state=42)
clusters = kmeans.fit_predict(category_encoded)
data['Cluster'] = clusters
# 클러스터별 카드 개수 계산
card_count_per_cluster = data.groupby('Cluster').size()
print(card_count_per_cluster)

Cluster
0    253
1     27
2     25
dtype: int64


In [ ]:
import pandas as pd
from kmodes.kmodes import KModes
from sklearn.preprocessing import MultiLabelBinarizer
# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/CARD-LANGCHAIN/data/crolling_final.csv')

# 'Category_List' 열 생성: 'Benefits'에서 'Category' 정보 추출 및 '언제나할인' 제거
data['Category_List'] = data['Benefits'].apply(
    lambda x: [item for item in eval(x)[0]['Category'] if item != '언제나할인']
    if isinstance(x, str) and eval(x) and 'Category' in eval(x)[0] else []
)
# 빈 리스트가 있는 행 제거
data = data[data['Category_List'].map(len) > 0]
# MultiLabelBinarizer를 사용해 'Category_List'를 One-hot 인코딩
mlb = MultiLabelBinarizer()
category_encoded = mlb.fit_transform(data['Category_List'])
# K-mode 클러스터링 수행 (예: 4개 클러스터로 설정)
kmode = KModes(n_clusters=4, init="Cao", n_init=1, verbose=1, random_state=42)
clusters = kmode.fit_predict(category_encoded)
# 클러스터 결과를 데이터프레임에 추가
data['Cluster'] = clusters
# 각 클러스터별 카드 개수 확인
card_count_per_cluster = data['Cluster'].value_counts()
print("Cluster별 카드 개수:\n", card_count_per_cluster)
# 클러스터별로 혜택 분포 요약
cluster_summary = data.groupby('Cluster')['Category_List'].apply(lambda x: x.explode().value_counts()).unstack().fillna(0)
print("Cluster별 혜택 요약:\n", cluster_summary)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 9, cost: 703.0
Run 1, iteration: 2/100, moves: 10, cost: 690.0
Run 1, iteration: 3/100, moves: 9, cost: 681.0
Run 1, iteration: 4/100, moves: 2, cost: 681.0
Cluster별 카드 개수:
 Cluster
0    105
3     31
1     27
2      3
Name: count, dtype: int64
Cluster별 혜택 요약:
          Priority Pass  간편결제  경차유류환급  관리비    교육  국민행복카드  그린카드    금융  납부 혜택  \
Cluster                                                                      
0                  3.0  13.0     2.0  7.0   4.0     1.0   2.0   7.0    8.0   
1                  0.0  16.0     0.0  2.0  26.0     1.0   0.0  10.0    1.0   
2                  0.0   1.0     0.0  3.0   3.0     1.0   0.0   3.0    0.0   
3                  0.0   5.0     0.0  2.0   2.0     0.0   0.0   1.0    0.0   

         대중교통  ...    의료    주유  체크카드겸용  카페/베이커리    통신   편의점  포인트/캐시백  프리미엄  \
Cluster        ...                                                           
0 

In [ ]:
cluster_summary

,간편결제,관리비,교육,납부 혜택,대중교통,대형마트,레저,문화,반려동물,뷰티,...,외식,육아,의료,주유,카페/베이커리,통신,편의점,포인트/캐시백,하이패스,항공마일리지
Cluster,,,,,,,,,,,,,,,,,,,,,
0,14.0,9.0,6.0,8.0,16.0,16.0,12.0,13.0,1.0,4.0,...,13.0,0.0,8.0,30.0,23.0,18.0,8.0,43.0,2.0,22.0
1,16.0,2.0,27.0,1.0,18.0,28.0,25.0,23.0,9.0,25.0,...,28.0,20.0,25.0,27.0,27.0,27.0,25.0,14.0,3.0,1.0
2,5.0,3.0,2.0,0.0,18.0,6.0,7.0,17.0,1.0,4.0,...,20.0,0.0,4.0,7.0,26.0,18.0,24.0,5.0,1.0,0.0


In [ ]:
0에 속한 카드?
어떤 카드를 뽑을 거니?
